<a href="https://colab.research.google.com/github/imabari/hyougo/blob/master/hyougo_excel_check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 検査陽性者の状況

In [0]:
import pandas as pd
import datetime

In [0]:
# ファイルをダウンロード
!wget -t 0 https://web.pref.hyogo.lg.jp/kk03/documents/yousei.xlsx -O yousei.xlsx

In [0]:
df0 = pd.read_excel("yousei.xlsx", index_col="発表年月日")

In [0]:
df0.columns = df0.columns.map(lambda s: s.replace("（累計）", "").strip())

In [0]:
df0.to_csv("yousei.tsv", sep="\t")

## 検査陽性者の状況（EXCEL）

In [0]:
df0.tail(3)

In [0]:
df0.drop("発表時間", axis=1, inplace=True)

# PCR検査数

In [0]:
# ファイルをダウンロード
!wget -t 0 https://web.pref.hyogo.lg.jp/kk03/documents/pcr.xlsx -O pcr.xlsx

In [0]:
df1 = pd.read_excel("pcr.xlsx", index_col="年月日").fillna(0).astype(int)

In [0]:
df1.to_csv("pcr.tsv", sep="\t")

## PCR検査日別件数（直近10件）

+ 最新日付を確認
+ 陽性患者数を確認
+ 検査実施数を確認

In [0]:
# 日別
df1.tail(10)

## ※PCR検査累計件数（直近10件）

+ 最新日付を確認
+ 陽性患者数を確認
+ 検査実施数を確認

In [0]:
# 累計
df1.cumsum().tail(10)

## PCR検査累計と陽性累計の確認

+ yousei.xlsxとpcr.xlsの最終行をチェック

In [0]:
df0.tail(1)

In [0]:
df1.cumsum().tail(1)

# 陽性患者情報

In [0]:
# ファイルをダウンロード
!wget -t 0 https://web.pref.hyogo.lg.jp/kk03/documents/corona-kanjajokyou.xlsx -O kanja.xlsx

In [0]:
df_head = pd.read_excel("kanja.xlsx", header=None, skiprows=3).dropna(how="all", axis=1)

In [0]:
df_head.to_csv("kanaja.tsv", index=False, header=False, sep="\t")

In [0]:
df_head.columns = ["".join(i).strip() for i in df_head.head(2).fillna("").T.values]

In [0]:
df_tmp = df_head.iloc[2:, :].copy().reset_index(drop=True)

In [0]:
df_tmp.fillna("")

In [0]:
df2 = df_tmp[df_tmp["番号"].notnull()].copy()

In [0]:
# 空列を削除
df2.dropna(how="all", axis=1, inplace=True)

In [0]:
df2.columns = df2.columns.map(lambda s: s.replace("\n", ""))

In [0]:
df2["年代"] = df2["年代"].replace("10歳未満", "0")

In [0]:
df2["番号"] = df2["番号"].astype("int64")
df2["年代"] = df2["年代"].astype("int64")

In [0]:
df2["発表日"] = df2["発表日"].apply(lambda date: pd.to_datetime(date, unit='D', origin=pd.Timestamp('1899/12/30')))

In [0]:
df2["備考欄"] = df2["備考欄"].str.replace("\n", "")

In [0]:
df2.set_index("番号", inplace=True)

## ※陽性患者数集計（累計）

+ 陽性患者の属性の累計を確認

In [0]:
len(df2)

## ※陽性患者数集計（日別）

+ 最新日付を確認
+ 陽性患者数を確認

In [0]:
df2["発表日"].value_counts().sort_index().reset_index().tail(10)

In [0]:
# 確認用
df1.tail(10)

## 陽性患者の属性（20件）

+ Noを最大値を確認（陽性患者数の合計と一致）
+ 陽性患者の属性を確認

In [0]:
# 降順の場合
df2.loc[:, ["発表日", "居住地", "年代", "性別", "備考欄"]].head(20)

## ※年代集計

+ 年代別陽性患者数の確認

In [0]:
df2["年代"].value_counts().sort_index().reset_index()

## クラスタ確認（Excelより抽出）

+ クラスター別陽性患者数の確認

In [0]:
df3 = df_tmp[df_tmp["番号"].isnull()].copy()

# 空列を削除
df3.dropna(how='all', axis=1, inplace=True)

In [0]:
df3.fillna("")

## ※クラスタ集計

+ クラスター別陽性患者数の確認

In [0]:
# クラスタのみを抽出
df4 = df2.loc[:, "認定こども園":].copy()

# クラスタ別にカウント
df4.notnull().sum().reset_index()

In [0]:
# クラスタ別患者数
df4.notnull().sum().sum()

## 該当なし

In [0]:
df4.isnull().all(axis=1).sum()

## ※重複者集計

+ クラスタ別陽性患者数の確認

In [0]:
# クラスタの〇の数をカウントして2箇所以上の人をカウント
duplication = df4.notnull().sum(axis=1) > 1
duplication.sum()